In [75]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import pickle

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)

INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_0:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_1:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module_1/Embeddings_en/sharded_10:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module_1/Embeddings_en/s

INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_0/self_attention/multihead_attention/q/kernel:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_0/self_attention/multihead_attention/q/kernel
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_0/self_attention/multihead_attention/v/kernel:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_0/self_attention/multihead_attention/v/kernel
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/ffn/conv1/bias:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/

INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_2/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_2/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_2/self_attention/multihead_attention/k/kernel:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_2/self_attention/multihead_attention/k/kernel
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_2/self_attention/multihead_attention/output_tr

INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/ffn/layer_prepostprocess/layer_norm/layer_norm_bias:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/ffn/layer_prepostprocess/layer_norm/layer_norm_bias
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/ffn/layer_prepostprocess/layer_norm/layer_norm_scale:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/ffn/layer_prepostprocess/layer_norm/layer_norm_scale
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/self_attention/layer_prepostprocess/layer_norm/lay

INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_prepostprocess/layer_norm/layer_norm_scale:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_prepostprocess/layer_norm/layer_norm_scale
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/dense/bias:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/dense/bias
INFO:tensorflow:Initialize variable module_1/Encoder_en/Transformer/dense/kernel:0 from checkpoint b'/var/folders/89/c5_6w72s6rjcw31hx74csxpc0000gp/T/tfhub_modules/96e8f1d3d4d90ce86b2db128249eb8143a91db73/variables/variables' with Encoder_en/Transformer/dense/kernel
INFO:tensorflow:Initialize variable module_1/Encoder_en/hidden_layers/t

In [12]:
test_image_caption = pickle.load(open('test_image_caption.pkl', 'rb'))

captions = list(map(lambda x: x['caption'], test_image_caption))

"""
test image:
[{"name": , "caption": , "vector": }]
"""
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(captions))
    print(message_embeddings.shape)
    new_test_image_vector = []
    for i, v in enumerate(test_image_caption):
        obj = {}
        obj['name'] = v['name']
        obj['caption'] = v['caption']
        obj['vector'] = message_embeddings[i]
        new_test_image_vector.append(obj)

    with open('test_image_vector.pkl', 'wb') as f:
        pickle.dump(new_test_image_vector, f)

(2000, 512)


In [103]:
from scipy.spatial.distance import cdist
import nltk
from tqdm import tqdm
import csv
import numpy as np
from pprint import pprint 

"""
test_image_pkl: (ordered)

[{"id": 1,
  "name": "1.jpg",
  "caption": "<string>",
  "caption_tags:": [],
  "tags": [],
  "vector": ,}
]


test_description_vector.pkl: (ordered)

[{"id": 1,
  "name": "1.txt",
  "description": ,
  "vector": each one has one line,
  "nouns": [...]}]


Algo:
- intersaction-union score, return a ranking, ordered by overlapped tags number.
- sentense vector similarity, return a ranking.


Improvement:
- preprocess the description (remove stop words; stemming; ), then turn into sentence vector.

- increase the test image tag by adding google vision api result to list. 

"""
test_image_vectors = pickle.load(open('test_image_vector.pkl', 'rb'))
test_description_vector = pickle.load(open('test_description_vector.pkl', 'rb'))


def view_test_image_pkl():
    new_v = test_image_vectors
    for v in new_v:
        v.pop("vector", None)
    pprint(new_v)

view_test_image_pkl()
    
def view_test_descrioption_vector_pkl():
    pprint(test_description_vector[:2])


def update_test_image_pkl(test_image_vectors):
    """
    To update test_image_vector.pkl.
    """
#     test_image_tag_base = "./all/data/tags_test/"
#     for d in test_image_vectors:
#         index = d["name"].split(".")[0]
#         d["id"] = index
#         tag_file = test_image_tag_base + index + ".txt"
#         with open(tag_file) as f:
#             content = f.readlines()
#             content = [x.strip().split(":")[1].split(" ") for x in content]
#             content = [item for sublist in content for item in sublist]
#         print(content)
#         d['tags'] = content
    for d in test_image_vectors:
        captions_tags = set()
        nouns = [word for (word, pos) in nltk.pos_tag(nltk.word_tokenize(d["caption"])) if pos[0] == 'N']
        for n in nouns:
            captions_tags.add(n)
        d["caption_tags"] = captions_tags
        
    test_image_vectors = sorted(test_image_vectors, key=lambda x: int(x['id']))
    with open('test_image_vector.pkl', 'wb') as f:
        pickle.dump(test_image_vectors, f)

        
# update_test_image_pkl(test_image_vectors)
# view_test_image_pkl()

        
def update_test_description_vector(test_description_vector):
    """
    To update test_description_vector.pkl.
    """
    for d in test_description_vector:
        all_nouns = set()
        for l in d["description"]:
            nouns = [word for (word, pos) in nltk.pos_tag(nltk.word_tokenize(l)) if pos[0] == 'N']
            for n in nouns:
                all_nouns.add(n)
        d["nouns"] = list(all_nouns)
#     descriptin_test_base = "./all/data/descriptions_test/"
#     for d in test_description_vector:
#         file_name = descriptin_test_base + d["name"]
#         with open(file_name) as f:
#             content = f.readlines()
#             content = [x.strip() for x in content] 
#         d["description"] = content
    test_description_vector = sorted(test_description_vector, key=lambda x: int(x['id']))
    with open('test_description_vector.pkl', 'wb') as f:
        pickle.dump(test_description_vector, f)

# update_test_description_vector(test_description_vector)
# view_test_descrioption_vector_pkl()
        
def calculate_dist_v1():
    """
    Using universal sentence vector for comparison.
    """
    images_vectors = list(map(lambda x: x['vector'], test_image_vectors))
    print("test images vector shape:", len(images_vectors))
    description_vectors = list(map(lambda x: x['vector'], test_description_vector))
    print("test description vector shape:", len(description_vectors))
    dist = cdist(description_vectors, images_vectors, 'cosine')
    print("description * images dist matrix, shape:", dist.shape)
    sorted_id = np.argsort(dist) # dist: N_description * N_images dist matrix.
    return sorted_id

def calculate_dist_v2(test_image_vectors, test_description_vector):
    """
    Rank by tag overlapping. 
    """
    dist = []
    for d in tqdm(test_description_vector):
        d_score = []
        for i in test_image_vectors:
            test_img_all_tags = list(i["caption_tags"]) + list(i["tags"])
            description_tags = d["nouns"]
            i_score = len(set(test_img_all_tags) & set(description_tags))
            d_score.append(i_score * (-1)) # becaues of argsort will sort increasing. 
        dist.append(d_score)
#     print(dist)
    print("description * images dist matrix, shape:", np.asarray(dist).shape)
    sorted_id = np.argsort(dist) # dist: N_description * N_images dist matrix.
    return sorted_id

def calculate_dist_v3(test_image_vectors, test_description_vector):
    """
    Ensemble v1 and v2. Find overlap, position on the front, and concat the v2 latter part to finish remaining. 
    """
    v1_ids = calculate_dist_v1()
    v2_ids = calculate_dist_v2(test_image_vectors, test_description_vector)
    new_ids = []
    for i, row_v1 in enumerate(v1_ids):
        row_v2 = v2_ids[i]
        overlap = list(set(row_v1[:30]) & set(row_v2[:30]))
        if len(overlap) >= 20:
            new_ids.append(overlap[:20])
        else:
            new_row = [*overlap, *row_v2]
            new_ids.append(new_row[:20])
#     print(new_ids)
    return np.asarray(new_ids)

[{'caption': 'a living room with a couch , a television , and a television .',
  'caption_tags': {'room', 'television', 'couch', 'living'},
  'id': '0',
  'name': '0.jpg',
  'tags': ['bed', 'backpack', 'suitcase', 'tie']},
 {'caption': 'a cow is standing in a field of grass .',
  'caption_tags': {'field', 'grass', 'cow'},
  'id': '1',
  'name': '1.jpg',
  'tags': ['cow']},
 {'caption': 'a man is playing a video game on a television .',
  'caption_tags': {'video', 'man', 'television', 'game'},
  'id': '2',
  'name': '2.jpg',
  'tags': ['frisbee', 'person']},
 {'caption': 'a street sign with a sign on the side of it .',
  'caption_tags': {'street', 'sign', 'side'},
  'id': '3',
  'name': '3.jpg',
  'tags': ['car', 'traffic', 'light', 'truck']},
 {'caption': 'a cat laying on a bed next to a cat .',
  'caption_tags': {'bed', 'cat'},
  'id': '4',
  'name': '4.jpg',
  'tags': ['cat', 'bed']},
 {'caption': 'a plate of food with a spoon and a fork .',
  'caption_tags': {'plate', 'food', 'fork'

  'id': '179',
  'name': '179.jpg',
  'tags': ['bottle',
           'dining',
           'table',
           'fork',
           'broccoli',
           'scissors',
           'sandwich',
           'carrot']},
 {'caption': 'a clock tower with a clock on top of it .',
  'caption_tags': {'tower', 'clock', 'top'},
  'id': '180',
  'name': '180.jpg',
  'tags': ['clock']},
 {'caption': 'a young girl is sitting on a bed with a cat .',
  'caption_tags': {'girl', 'bed', 'cat'},
  'id': '181',
  'name': '181.jpg',
  'tags': ['person', 'donut', 'couch']},
 {'caption': 'a table with a bunch of food on it',
  'caption_tags': {'bunch', 'food', 'table'},
  'id': '182',
  'name': '182.jpg',
  'tags': ['bowl', 'scissors']},
 {'caption': 'a giraffe standing in a field next to a tree .',
  'caption_tags': {'standing', 'field', 'tree', 'giraffe'},
  'id': '183',
  'name': '183.jpg',
  'tags': ['giraffe']},
 {'caption': 'a street sign with a sign on top of it .',
  'caption_tags': {'street', 'top', 'sign'}

  'caption_tags': {'man', 'snow', 'snowboard'},
  'id': '351',
  'name': '351.jpg',
  'tags': ['skateboard', 'person', 'backpack']},
 {'caption': 'a room with a bed and a window in the background .',
  'caption_tags': {'room', 'bed', 'window', 'background'},
  'id': '352',
  'name': '352.jpg',
  'tags': ['couch', 'chair']},
 {'caption': 'a woman holding a pink umbrella in her hand .',
  'caption_tags': {'woman', 'umbrella', 'hand', 'pink'},
  'id': '353',
  'name': '353.jpg',
  'tags': ['umbrella', 'person', 'backpack']},
 {'caption': 'a man is skiing down a hill in the snow .',
  'caption_tags': {'hill', 'man', 'snow'},
  'id': '354',
  'name': '354.jpg',
  'tags': ['person', 'skateboard']},
 {'caption': 'a plate of food with a fork and a fork .',
  'caption_tags': {'plate', 'food', 'fork'},
  'id': '355',
  'name': '355.jpg',
  'tags': ['dining', 'table', 'cup', 'donut']},
 {'caption': 'a person is sitting on a table with a laptop .',
  'caption_tags': {'laptop', 'person', 'table'},


 {'caption': 'a sandwich with a sandwich and a salad on a plate .',
  'caption_tags': {'plate', 'salad', 'sandwich'},
  'id': '538',
  'name': '538.jpg',
  'tags': ['hot', 'dog', 'bowl', 'cup']},
 {'caption': 'a group of cars parked on a street next to a street .',
  'caption_tags': {'street', 'cars', 'group'},
  'id': '539',
  'name': '539.jpg',
  'tags': ['car', 'truck', 'traffic', 'light', 'person']},
 {'caption': 'a man standing in front of a building with a clock .',
  'caption_tags': {'man', 'front', 'clock', 'building'},
  'id': '540',
  'name': '540.jpg',
  'tags': ['chair', 'person', 'remote', 'tv', 'backpack', 'bottle']},
 {'caption': 'a woman walking down a sidewalk holding an umbrella .',
  'caption_tags': {'woman', 'umbrella', 'sidewalk'},
  'id': '541',
  'name': '541.jpg',
  'tags': ['umbrella', 'person', 'bench']},
 {'caption': 'a person holding a cell phone in front of a wall .',
  'caption_tags': {'wall', 'phone', 'person', 'front', 'cell'},
  'id': '542',
  'name': '

 {'caption': 'a man and a woman are sitting on a bench .',
  'caption_tags': {'woman', 'man', 'bench'},
  'id': '745',
  'name': '745.jpg',
  'tags': ['dog', 'person', 'frisbee']},
 {'caption': 'a cat is sitting on a couch with a cat .',
  'caption_tags': {'cat'},
  'id': '746',
  'name': '746.jpg',
  'tags': ['cat']},
 {'caption': 'a man is sitting on a surfboard in the ocean .',
  'caption_tags': {'surfboard', 'man', 'ocean'},
  'id': '747',
  'name': '747.jpg',
  'tags': ['person', 'surfboard', 'umbrella']},
 {'caption': 'a clock tower with a clock on top of it .',
  'caption_tags': {'tower', 'clock', 'top'},
  'id': '748',
  'name': '748.jpg',
  'tags': ['clock']},
 {'caption': 'a man riding a bike down a street next to a street .',
  'caption_tags': {'street', 'man', 'bike'},
  'id': '749',
  'name': '749.jpg',
  'tags': ['bicycle', 'car', 'person', 'fire', 'hydrant', 'truck', 'bus']},
 {'caption': 'a man is surfing on a wave in the ocean .',
  'caption_tags': {'man', 'wave', 'oce

...

 {'caption': 'a man is standing in a kitchen with a refrigerator .',
  'caption_tags': {'man', 'kitchen', 'refrigerator'},
  'id': '1284',
  'name': '1284.jpg',
  'tags': ['bottle',
           'refrigerator',
           'person',
           'bowl',
           'apple',
           'oven',
           'sink']},
 {'caption': 'a zebra is standing in the grass near a tree .',
  'caption_tags': {'tree', 'grass', 'zebra'},
  'id': '1285',
  'name': '1285.jpg',
  'tags': ['zebra']},
 {'caption': 'a woman is holding a umbrella in the sand .',
  'caption_tags': {'woman', 'umbrella', 'sand'},
  'id': '1286',
  'name': '1286.jpg',
  'tags': ['boat', 'person', 'umbrella']},
 {'caption': 'a person is sitting on a bench with a laptop .',
  'caption_tags': {'laptop', 'bench', 'person'},
  'id': '1287',
  'name': '1287.jpg',
  'tags': ['person', 'skateboard']},
 {'caption': 'a man is surfing on a wave in the ocean .',
  'caption_tags': {'man', 'wave', 'ocean'},
  'id': '1288',
  'name': '1288.jpg',
  'ta

  'tags': ['bird', 'giraffe']},
 {'caption': 'a man is playing frisbee in a park .',
  'caption_tags': {'man', 'frisbee', 'park'},
  'id': '1433',
  'name': '1433.jpg',
  'tags': ['person', 'frisbee']},
 {'caption': 'a man is playing frisbee in a field .',
  'caption_tags': {'field', 'man', 'frisbee'},
  'id': '1434',
  'name': '1434.jpg',
  'tags': ['sports', 'ball', 'person']},
 {'caption': 'a display of various types of bananas .',
  'caption_tags': {'display', 'types', 'bananas'},
  'id': '1435',
  'name': '1435.jpg',
  'tags': ['bottle', 'vase', 'bowl']},
 {'caption': 'a baseball player swinging a bat at a ball',
  'caption_tags': {'bat', 'player', 'baseball', 'ball'},
  'id': '1436',
  'name': '1436.jpg',
  'tags': ['sports', 'ball', 'person', 'baseball', 'bat', 'baseball', 'glove']},
 {'caption': 'a clock on a pole with a clock on it .',
  'caption_tags': {'pole', 'clock'},
  'id': '1437',
  'name': '1437.jpg',
  'tags': []},
 {'caption': 'a group of cows in a field with a large

 {'caption': 'a man is skiing down a hill in the snow .',
  'caption_tags': {'hill', 'man', 'snow'},
  'id': '1607',
  'name': '1607.jpg',
  'tags': ['skis', 'person']},
 {'caption': 'a laptop computer sitting on a wooden table .',
  'caption_tags': {'computer', 'table'},
  'id': '1608',
  'name': '1608.jpg',
  'tags': ['cell', 'phone', 'book', 'person', 'clock']},
 {'caption': 'a horse pulling a horse on a dirt road .',
  'caption_tags': {'dirt', 'road', 'horse'},
  'id': '1609',
  'name': '1609.jpg',
  'tags': ['dog', 'horse', 'person']},
 {'caption': 'a young boy is holding a bat in a field .',
  'caption_tags': {'field', 'bat', 'boy'},
  'id': '1610',
  'name': '1610.jpg',
  'tags': ['sports', 'ball', 'person', 'baseball', 'bat']},
 {'caption': 'a bus that is driving down a street .',
  'caption_tags': {'street', 'bus'},
  'id': '1611',
  'name': '1611.jpg',
  'tags': ['car', 'bus', 'person', 'traffic', 'light']},
 {'caption': 'a close up of a plate of food with a banana',
  'capti

 {'caption': 'a man holding a tennis racquet on a tennis court .',
  'caption_tags': {'tennis', 'man', 'court', 'racquet'},
  'id': '1813',
  'name': '1813.jpg',
  'tags': ['person', 'car', 'sports', 'ball', 'tennis', 'racket']},
 {'caption': 'a bunch of oranges sitting on a table .',
  'caption_tags': {'bunch', 'table', 'oranges'},
  'id': '1814',
  'name': '1814.jpg',
  'tags': ['orange']},
 {'caption': 'a man is skiing down a hill in the snow .',
  'caption_tags': {'hill', 'man', 'snow'},
  'id': '1815',
  'name': '1815.jpg',
  'tags': ['person', 'skis']},
 {'caption': 'a man and woman are sitting in front of a car .',
  'caption_tags': {'woman', 'man', 'front', 'car'},
  'id': '1816',
  'name': '1816.jpg',
  'tags': ['person', 'cell', 'phone', 'bicycle']},
 {'caption': 'a group of people standing around a red bus .',
  'caption_tags': {'people', 'bus', 'group'},
  'id': '1817',
  'name': '1817.jpg',
  'tags': ['truck', 'person']},
 {'caption': 'a group of people riding on a motorcy

  'tags': ['sheep', 'person', 'backpack']},
 {'caption': 'a large group of boats in the water',
  'caption_tags': {'boats', 'group', 'water'},
  'id': '1996',
  'name': '1996.jpg',
  'tags': ['boat', 'person']},
 {'caption': 'a train is on a track near a building .',
  'caption_tags': {'building', 'train', 'track'},
  'id': '1997',
  'name': '1997.jpg',
  'tags': ['boat', 'bench']},
 {'caption': 'a large airplane is parked on a runway .',
  'caption_tags': {'runway', 'airplane'},
  'id': '1998',
  'name': '1998.jpg',
  'tags': ['airplane']},
 {'caption': 'a pizza sitting on a table next to a glass of wine .',
  'caption_tags': {'pizza', 'glass', 'wine'},
  'id': '1999',
  'name': '1999.jpg',
  'tags': ['fork', 'knife', 'bowl', 'pizza', 'dining', 'table']}]


In [100]:
def main():
    with open('submission_v3.csv', 'w') as csvfile:
        # write csv header
        fieldnames = ['Descritpion_ID', "Top_20_Image_IDs"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

#         sorted_id = calculate_dist_v1()
        sorted_id = calculate_dist_v3(test_image_vectors, test_description_vector) # return N_description * N_images dist matrix.
        
        for i, row in enumerate(sorted_id):
            top_choices =  list(map(lambda x: test_image_vectors[x]['name'], row[:20]))
    #         print(top_choices)
            res = {}
            res['Descritpion_ID'] = test_description_vector[i]['name'] # file name
            res['Top_20_Image_IDs'] = " ".join(top_choices)
            writer.writerow(res)

    print("Writing Complete.")

main()



"""
# get the nouns of the sentence[five descriptions].
# the images with the correct tag will form first group. others forms the other group. 
"""
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
def visualize():
    """
    Visualize part. 
    """
    test_images_base = "./all/data/images_test/"
    for f in top_choices:
        f_path = test_images_base + f
        img = mpimg.imread(f_path)
        plt.figure()
        plt.imshow(img)
        plt.show()

test images vector shape: 2000
test description vector shape: 2000




  0%|          | 0/2000 [00:00<?, ?it/s]

description * images dist matrix, shape: (2000, 2000)




  1%|▏         | 26/2000 [00:00<00:07, 255.34it/s]

  2%|▎         | 50/2000 [00:00<00:07, 249.90it/s]

  4%|▍         | 76/2000 [00:00<00:07, 251.20it/s]

  5%|▌         | 103/2000 [00:00<00:07, 254.59it/s]

  6%|▌         | 123/2000 [00:00<00:08, 218.70it/s]

  7%|▋         | 148/2000 [00:00<00:08, 225.86it/s]

  9%|▊         | 171/2000 [00:00<00:08, 225.98it/s]

 10%|▉         | 192/2000 [00:00<00:09, 183.78it/s]

 11%|█         | 211/2000 [00:01<00:11, 160.04it/s]

 11%|█▏        | 228/2000 [00:01<00:12, 142.75it/s]

 12%|█▏        | 244/2000 [00:01<00:12, 138.68it/s]

 13%|█▎        | 259/2000 [00:01<00:12, 137.47it/s]

 14%|█▎        | 274/2000 [00:01<00:12, 136.11it/s]

 14%|█▍        | 288/2000 [00:01<00:12, 134.29it/s]

 15%|█▌        | 305/2000 [00:01<00:11, 142.10it/s]

 16%|█▋        | 329/2000 [00:01<00:10, 161.74it/s]

 18%|█▊        | 354/2000 [00:01<00:09, 179.37it/s]

 19%|█▉        | 379/2000 [00:02<00:08, 194.26it/s]

 20%|██        | 404/2000 [00:02<00:07, 206.51i

description * images dist matrix, shape: (2000, 2000)
[[1131, 93, 742, 93, 484, 878, 1521, 50, 1258, 124, 36, 1816, 1131, 742, 1228, 1045, 1231, 1394, 1380, 1729], [833, 1741, 1199, 495, 753, 1880, 154, 1917, 158, 1880, 758, 1741, 1753, 457, 1742, 154, 158, 238, 589, 1812], [979, 545, 539, 115, 1665, 1512, 663, 1955, 511, 42, 1124, 505, 649, 1627, 159, 838, 3, 184, 765, 1866], [1187, 71, 1748, 1620, 1052, 161, 1052, 669, 1418, 1453, 1620, 573, 1748, 1053, 732, 71, 72, 1765, 471, 871], [1394, 292, 942, 878, 1231, 68, 1311, 36, 1862, 1816, 312, 912, 339, 1394, 942, 1258, 292, 745, 388, 725], [1696, 1923, 389, 1402, 1355, 1292, 1837, 526, 814, 1296, 49, 274, 51, 846, 492, 370, 1335, 1594, 1943, 492], [712, 489, 1768, 1265, 1815, 379, 1151, 33, 156, 1734, 429, 908, 391, 910, 1265, 899, 1940, 535, 1275, 583], [136, 689, 484, 528, 1249, 50, 273, 1904, 1521, 1380, 1277, 1285, 1769, 1119, 136, 773, 1804, 540, 1155, 975], [781, 1037, 260, 749, 1242, 103, 1543, 1570, 561, 1443, 113, 114, 1901, 3

In [51]:
"""
This section dedicated to create the sentence vector for each test description.z 

test_description_vector.pkl:

[{"name": "1.txt", "description": , "vector": each one has one line}]
"""
descriptin_test_base = "./all/data/descriptions_test/"
description_lst = []
test_description_vector = []
for file in tqdm(os.listdir(descriptin_test_base)):
    if file.endswith(".txt"):
        file_name = descriptin_test_base + file
        with open(file_name) as f:
            content = f.readlines()
            content = [x.strip() for x in content] 
        print(content[-1])
        description_lst.append(content[-1])
        obj = {}
        obj['name'] = file
        obj['description'] = content[-1]
        test_description_vector.append(obj)
        
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embedding = session.run(embed(description_lst)) # parallalize!!


for i, v in tqdm(enumerate(message_embedding)):
    test_description_vector[i]['vector'] = v

with open('test_description_vector.pkl', 'wb') as f:
    pickle.dump(test_description_vector, f)



  0%|          | 0/2000 [00:00<?, ?it/s]

  8%|▊         | 169/2000 [00:00<00:01, 1686.73it/s]

 20%|██        | 404/2000 [00:00<00:00, 1840.18it/s]

A giraffe sticking its tongue out over a wooden fence.
A boy holds the guitar controller from Guitar Hero.
A pair of scissors sticking into a table.
A freight train is running through the country.
People flying kites in a park on a windy day.
This is a birthday cake for someones first birthday.
a couple of people that are looking at a cake
A pizza with toppings being put in the oven.
I am unable to see the image above.
A dog with a pink bow on head, leaning over a couch in livingroom.
We see a street sign with houses in the background.
A colorful display is set up for a school book fair.
Several people in a kitchen and one is standing at a sink.
a train on a track near many trees
An old truck in the field with three men
a close up of three hot dogs with mustard
A bed in the corner of a room has a puffy white comforter on it.
A boy and a girl play soccer together on the soccer field.
a close up of a door with two dogs in the glass of either side of the door
A man jumping up to catch a f



 31%|███       | 619/2000 [00:00<00:00, 1921.60it/s]

 44%|████▍     | 882/2000 [00:00<00:00, 2090.04it/s]

A two story gym sitting next to a traffic signal.
A nicely decorated kitchen with black and white theme.
A close-up of a pile of glazed doughnuts.
flowers in a pot sitting on a cement wall
A small kitchen with a table with two chairs.
A statue of clowns riding a tandem bike,
A young man on a skateboard grinding the edge of a concrete ledge.
four tennis players on a tennis court playing doubles tennis
The church is up on the hill in the country.
boy in a sports jersey holds skateboard in right hand
A stock photo shows horses and their riders
A white automatic dishwasher sitting under a microwave oven.
Seven oranges stacked together in a white bowl.
An insect is chewing some carrot inside a wooden cage.
a man skiis across a field in the snow
A large dog with blue eyes on a couch.
Blue truck rolling down the side of a city street.
A guinea pig and a dog look at each other on a bed.
The bus is parked next to the curb.
A flat screen tv in a living room with several windows.
a tennis player 



 58%|█████▊    | 1162/2000 [00:00<00:00, 2261.07it/s]

 73%|███████▎  | 1461/2000 [00:00<00:00, 2438.07it/s]

A person putting some food into the oven.
A red toothbrush next to a bottle of Crest toothpaste.
A man in a wheelchair is next to a fire hydrant.
A man in a yellow shirt jumping over three people.
A man with different clothing on each half of his body
A giraffe and its young standing in an enclosure.
A partitioned tray holds several different lunch items.
A woman takes a picture of a blue train going into a tunnel.
A picture of a toy elephant on a table.
A white horse pulling a wagon of people .
A young man stands on a skateboard on a sidewalk.
A lunch spread with a sandwich and glass of wine.
There is a snow boarder going down a snowing hill.
A train is moving on tracks in an open field.
A plate containing a serving of broccoli, cooked meat, and sauteed mushrooms.
A living area with a sofa, coffee table and an entertainment unit.
PERSON IN BLACK AND WHITE PICTURE HOLDING HAIR DRYER
A different color train going under a bridge.
A picture of a teddy bear with some message
A red double d



 87%|████████▋ | 1747/2000 [00:00<00:00, 2550.51it/s]

100%|██████████| 2000/2000 [00:00<00:00, 2550.69it/s]

an alarm clock radio with an ipod, a cell phone and a book
A cat laying on a blanket on top of a pillow.
a living room full of chairs and decorations
Papers cover the windows of an old building.
A store front with a brightly colored sign in another language.
Baseball game close up of batter, catcher and baseball fans.
A man holding up a tooth brush as he yells at it.
A yellow fire hydrant with several valves on it.
A bicycle that is tied to a post along a sidewalk.
A pair of stuffed bears stand posed in front of a background.
A zebra is relaxing near the rocks on a nice day.
Pizza served at an Italian restaurant, with patrons in background.
An ostrich standing in a zoo pin near some zebras.
A person on a motorbike drives past a gate.
Two guys stand by bottom of stairs playing the Wii
Fruits and vegetables inside of a cooking pot.
A white sink sitting underneath a mirror in a bathroom.
Several wooden benches near water and a lighthouse.
A large slab of sponge cake sits upon a flowery pl



0it [00:00, ?it/s]

2000it [00:00, 106073.47it/s]